In [ ]:
import pickle
from datetime import datetime
from pathlib import Path

# clear cell output command
from IPython.display import clear_output

from beaks import construct_grains_dictionary

today = datetime.today().strftime("D-%Y-%m-%d-T-%H-%M")
print(today)

In [ ]:
base_dir = Path("/Users/sylvi/topo_data/beaks")
assert base_dir.exists()
beak_topo_data_dir = base_dir / "output-beaks-topostats-unet-good"
assert beak_topo_data_dir.exists()

hummingbird_dir = beak_topo_data_dir / "hummingbird/processed/"
assert hummingbird_dir.exists()
magpie_dir = beak_topo_data_dir / "magpie/processed/"
assert magpie_dir.exists()

# grab files in both directories ending in .topostats
hummingbird_files = hummingbird_dir.glob("*.topostats")
magpie_files = magpie_dir.glob("*.topostats")
# merge lists
all_files = list(hummingbird_files) + list(magpie_files)
print(f"Found {len(all_files)} files")

## construct grain dictionary

In [ ]:
grains_dictionary = construct_grains_dictionary(file_list=all_files, plot=False)
clear_output()
print(f"found {len(grains_dictionary)} grains in {len(all_files)} files")

# save the grain dictionary

In [ ]:
grains_with_beaks = [7, 13, 16, 26, 28, 30, 34, 36, 37, 38, 43, 45, 54, 55, 58, 65, 66, 71, 72, 79, 82, 89, 91, 92, 93]

grains_with_beaks_dictionary = {}

for grain_index in grains_with_beaks:
    grains_with_beaks_dictionary[grain_index] = grains_dictionary[grain_index]

# save the grains with beaks dictionary
grains_with_beaks_save_dir = base_dir / "grain-dictionaries"
assert grains_with_beaks_save_dir.exists()
with open(grains_with_beaks_save_dir / f"grains_with_beaks_{today}.pkl", "wb") as f:
    pickle.dump(grains_with_beaks_dictionary, f)